In [ ]:
!pip install unsloth==2025.9.6
!pip install --no-deps trl==0.8.6 xformers accelerate peft bitsandbytes

In [3]:
import unsloth
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Phi-3-mini-4k-instruct-bnb-4bit",  # Efficient classification model
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.9.6: Fast Mistral patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Standard rank for classification patterns
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
)

Unsloth 2025.9.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
from datasets import load_dataset
dataset = load_dataset("dair-ai/emotion", split="train[:5000]")

README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [7]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})

In [8]:
print(dataset['text'][100])

i wont let me child cry it out because i feel that loving her and lily when she was little was going to be opportunities that only lasted for those short few months


In [9]:
print(dataset['label'][100])

2


In [10]:
def format_emotion_classification(examples):
    """Format examples for emotion classification task"""
    texts = []

    # Emotion labels mapping
    emotion_labels = {
        0: "sadness",
        1: "joy",
        2: "love",
        3: "anger",
        4: "fear",
        5: "surprise"
    }

    for i in range(len(examples["text"])):
        text = examples["text"][i]
        emotion_id = examples["label"][i]
        emotion = emotion_labels[emotion_id]

        # Create emotion classification format
        prompt = f"Analyze the emotion expressed in this text and classify it as one of: sadness, joy, love, anger, fear, or surprise.\n\nText: {text}\n\nEmotion:"

        messages = [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": emotion}
        ]

        # Apply Phi-3 chat template
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,
        )

        texts.append(text)

    return {"text": texts}

In [11]:
formatted_dataset = dataset.map(
    format_emotion_classification,
    batched=True,
    remove_columns=dataset.column_names,
    desc="Formatting for emotion classification"
)

Formatting for emotion classification:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [13]:
formatted_dataset['text'][100]

'<|user|>\nAnalyze the emotion expressed in this text and classify it as one of: sadness, joy, love, anger, fear, or surprise.\n\nText: i wont let me child cry it out because i feel that loving her and lily when she was little was going to be opportunities that only lasted for those short few months\n\nEmotion:<|end|>\n<|assistant|>\nlove<|end|>\n<|endoftext|>'

In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    warmup_steps=10,
    max_steps=100,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=5,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=3407,
    output_dir="phi3_emotion_classifier",
    save_strategy="no",
)

In [15]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    args=training_args,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=False,
)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [16]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416 of 3,850,963,968 (0.78% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abeshith (abeshith-dr-m-g-r-educational-and-research-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
5,2.533300
10,1.949200
15,1.259800
20,1.134200
25,1.268700
30,1.037200
35,1.187700
40,0.990600
45,1.039300
50,1.046800


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=100, training_loss=1.2117786359786988, metrics={'train_runtime': 203.8436, 'train_samples_per_second': 3.925, 'train_steps_per_second': 0.491, 'total_flos': 1494350988779520.0, 'train_loss': 1.2117786359786988, 'epoch': 0.16})

In [17]:
model.save_pretrained("phi3_emotion_analyst")
tokenizer.save_pretrained("phi3_emotion_analyst")

('phi3_emotion_analyst/tokenizer_config.json',
 'phi3_emotion_analyst/special_tokens_map.json',
 'phi3_emotion_analyst/chat_template.jinja',
 'phi3_emotion_analyst/tokenizer.model',
 'phi3_emotion_analyst/added_tokens.json',
 'phi3_emotion_analyst/tokenizer.json')

In [ ]:
FastLanguageModel.for_inference(model)

In [19]:
def classify_emotion(text, analysis_type="emotion", max_tokens=50):
    """Classify emotion or sentiment in text"""

    prompts_by_type = {
        "emotion": f"Analyze the emotion expressed in this text and classify it as one of: sadness, joy, love, anger, fear, or surprise.\n\nText: {text}\n\nEmotion:",
        "sentiment": f"Analyze the sentiment of this text and classify it as: positive, negative, or neutral.\n\nText: {text}\n\nSentiment:",
        "mood": f"Identify the overall mood conveyed in this text. Choose from: happy, sad, excited, calm, frustrated, anxious, content, or confused.\n\nText: {text}\n\nMood:",
        "intensity": f"Rate the emotional intensity of this text on a scale: low, medium, or high.\n\nText: {text}\n\nIntensity:"
    }

    messages = [
        {"role": "user", "content": prompts_by_type.get(analysis_type, prompts_by_type["emotion"])}
    ]

    text_input = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer([text_input], return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.1,  # Low temp for consistent classification
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )

    # Extract generated tokens and decode
    generated_tokens = outputs[0][len(inputs['input_ids'][0]):]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return response.strip()

In [20]:
emotion_texts = [
    "I just got promoted at work! This is the best day ever!",
    "I can't believe they cancelled my favorite TV show. I'm so disappointed.",
    "Walking alone in the dark alley made my heart race with terror.",
    "Spending time with my family during the holidays fills my heart with warmth.",
    "The traffic is so bad today, I'm going to be late for my important meeting!",
    "Wow, I never expected to see you here! What a pleasant surprise!"
]

print("🎭 BASIC EMOTION CLASSIFICATION:")
for i, text in enumerate(emotion_texts, 1):
    print(f"\n📝 Text {i}: {text}")
    emotion = classify_emotion(text, "emotion", 30)
    print(f"🎯 Detected Emotion: {emotion}")
    print("-" * 60)

🎭 BASIC EMOTION CLASSIFICATION:

📝 Text 1: I just got promoted at work! This is the best day ever!
🎯 Detected Emotion: joy
------------------------------------------------------------

📝 Text 2: I can't believe they cancelled my favorite TV show. I'm so disappointed.
🎯 Detected Emotion: sadness
------------------------------------------------------------

📝 Text 3: Walking alone in the dark alley made my heart race with terror.
🎯 Detected Emotion: fear
------------------------------------------------------------

📝 Text 4: Spending time with my family during the holidays fills my heart with warmth.
🎯 Detected Emotion: joy
------------------------------------------------------------

📝 Text 5: The traffic is so bad today, I'm going to be late for my important meeting!
🎯 Detected Emotion: anger
------------------------------------------------------------

📝 Text 6: Wow, I never expected to see you here! What a pleasant surprise!
🎯 Detected Emotion: surprise
------------------------------

In [21]:
sentiment_texts = [
    "This product is absolutely amazing! Best purchase I've ever made.",
    "The customer service was terrible. They kept me waiting for hours.",
    "The weather is okay today, nothing special.",
    "I love how this app makes my daily tasks so much easier.",
    "The movie was disappointing. The plot made no sense at all.",
    "The restaurant has decent food and reasonable prices."
]

print("\n📊 SENTIMENT ANALYSIS:")
for i, text in enumerate(sentiment_texts, 1):
    print(f"\n📝 Text {i}: {text}")
    sentiment = classify_emotion(text, "sentiment", 20)
    print(f"📈 Sentiment: {sentiment}")
    print("-" * 60)


📊 SENTIMENT ANALYSIS:

📝 Text 1: This product is absolutely amazing! Best purchase I've ever made.
📈 Sentiment: positive
------------------------------------------------------------

📝 Text 2: The customer service was terrible. They kept me waiting for hours.
📈 Sentiment: negative
------------------------------------------------------------

📝 Text 3: The weather is okay today, nothing special.
📈 Sentiment: negative
------------------------------------------------------------

📝 Text 4: I love how this app makes my daily tasks so much easier.
📈 Sentiment: positive
------------------------------------------------------------

📝 Text 5: The movie was disappointing. The plot made no sense at all.
📈 Sentiment: negative
------------------------------------------------------------

📝 Text 6: The restaurant has decent food and reasonable prices.
📈 Sentiment: positive
------------------------------------------------------------


In [22]:
social_media_posts = [
    "Just finished my first marathon! 26.2 miles of pure determination! 💪 #achievement",
    "Another Monday morning... Why do weekends go by so fast? 😞",
    "Can't wait for the concert tonight! My favorite band is finally in town! 🎵",
    "Stuck in traffic again. This city's infrastructure is a joke. 😡",
    "Cuddling with my cat on this rainy Sunday. Perfect way to spend the afternoon. 🐱",
    "Job interview tomorrow. Nervous but excited about the opportunity! 🤞"
]

print("\n📱 SOCIAL MEDIA EMOTION ANALYSIS:")
for i, post in enumerate(social_media_posts, 1):
    print(f"\n📱 Post {i}: {post}")
    emotion = classify_emotion(post, "emotion", 25)
    mood = classify_emotion(post, "mood", 25)
    print(f"😊 Emotion: {emotion}")
    print(f"🎭 Mood: {mood}")
    print("-" * 60)


📱 SOCIAL MEDIA EMOTION ANALYSIS:

📱 Post 1: Just finished my first marathon! 26.2 miles of pure determination! 💪 #achievement
😊 Emotion: joy
🎭 Mood: excited
------------------------------------------------------------

📱 Post 2: Another Monday morning... Why do weekends go by so fast? 😞
😊 Emotion: sadness
🎭 Mood: sad
------------------------------------------------------------

📱 Post 3: Can't wait for the concert tonight! My favorite band is finally in town! 🎵
😊 Emotion: joy
🎭 Mood: excited
------------------------------------------------------------

📱 Post 4: Stuck in traffic again. This city's infrastructure is a joke. 😡
😊 Emotion: anger
🎭 Mood: frustrated
------------------------------------------------------------

📱 Post 5: Cuddling with my cat on this rainy Sunday. Perfect way to spend the afternoon. 🐱
😊 Emotion: joy
🎭 Mood: content
------------------------------------------------------------

📱 Post 6: Job interview tomorrow. Nervous but excited about the opportunity! 🤞
😊 Emo

In [25]:
customer_reviews = [
    "Outstanding service! The staff went above and beyond to help me find exactly what I needed.",
    "Product arrived damaged and customer support was unhelpful. Very frustrated with this experience.",
    "Average experience. Nothing exceptional but nothing terrible either.",
    "I'm absolutely thrilled with my purchase! Exceeded all my expectations.",
    "Delivery was delayed twice and no one bothered to inform me. Very poor communication.",
    "Good quality product at a fair price. Would recommend to others."
]

print("\n💬 CUSTOMER FEEDBACK ANALYSIS:")
for i, review in enumerate(customer_reviews, 1):
    print(f"\n💬 Review {i}: {review}")
    sentiment = classify_emotion(review, "sentiment", 20)
    intensity = classify_emotion(review, "intensity", 15)
    print(f"📊 Sentiment: {sentiment}")
    print(f"🔥 Intensity: {intensity}")
    print("-" * 60)


💬 CUSTOMER FEEDBACK ANALYSIS:

💬 Review 1: Outstanding service! The staff went above and beyond to help me find exactly what I needed.
📊 Sentiment: positive
🔥 Intensity: medium
------------------------------------------------------------

💬 Review 2: Product arrived damaged and customer support was unhelpful. Very frustrated with this experience.
📊 Sentiment: negative
🔥 Intensity: high
------------------------------------------------------------

💬 Review 3: Average experience. Nothing exceptional but nothing terrible either.
📊 Sentiment: negative
🔥 Intensity: low
------------------------------------------------------------

💬 Review 4: I'm absolutely thrilled with my purchase! Exceeded all my expectations.
📊 Sentiment: positive
🔥 Intensity: high
------------------------------------------------------------

💬 Review 5: Delivery was delayed twice and no one bothered to inform me. Very poor communication.
📊 Sentiment: negative
🔥 Intensity: high
------------------------------------------

In [26]:
wellness_texts = [
    "I've been feeling overwhelmed lately. Everything seems to be piling up and I can't catch a break.",
    "Meditation and yoga have really helped me find inner peace and balance in my life.",
    "Some days I feel like nothing I do matters. It's hard to find motivation to keep going.",
    "Grateful for all the support from my friends and family during this difficult time.",
    "I'm proud of how far I've come in my personal growth journey this year.",
    "Anxiety keeps me awake at night, wondering about all the things that could go wrong."
]

print("\n🧘 MENTAL HEALTH & WELLNESS ANALYSIS:")
for i, text in enumerate(wellness_texts, 1):
    print(f"\n🧘 Text {i}: {text}")
    emotion = classify_emotion(text, "emotion", 25)
    mood = classify_emotion(text, "mood", 25)
    print(f"💭 Emotion: {emotion}")
    print(f"🎯 Mood: {mood}")
    print("-" * 60)


🧘 MENTAL HEALTH & WELLNESS ANALYSIS:

🧘 Text 1: I've been feeling overwhelmed lately. Everything seems to be piling up and I can't catch a break.
💭 Emotion: sadness
🎯 Mood: anxious
------------------------------------------------------------

🧘 Text 2: Meditation and yoga have really helped me find inner peace and balance in my life.
💭 Emotion: joy
🎯 Mood: content
------------------------------------------------------------

🧘 Text 3: Some days I feel like nothing I do matters. It's hard to find motivation to keep going.
💭 Emotion: sadness
🎯 Mood: sad
------------------------------------------------------------

🧘 Text 4: Grateful for all the support from my friends and family during this difficult time.
💭 Emotion: joy
🎯 Mood: content
------------------------------------------------------------

🧘 Text 5: I'm proud of how far I've come in my personal growth journey this year.
💭 Emotion: joy
🎯 Mood: content
------------------------------------------------------------

🧘 Text 6: Anxiety

In [27]:
brand_mentions = [
    "Just tried the new @CoffeeShop blend and it's incredible! Perfect start to my morning.",
    "@TechCompany's latest update broke my favorite feature. Really disappointed right now.",
    "@FitnessApp helped me reach my goals! Lost 20 pounds thanks to their program.",
    "Had a terrible experience at @Restaurant yesterday. Food was cold and service was slow.",
    "@BookStore has an amazing selection and the staff is always so helpful and knowledgeable.",
    "Not sure how I feel about @SocialPlatform's new privacy policy. Seems concerning."
]

print("\n🏢 BRAND MENTION SENTIMENT ANALYSIS:")
for i, mention in enumerate(brand_mentions, 1):
    print(f"\n🏢 Mention {i}: {mention}")
    sentiment = classify_emotion(mention, "sentiment", 20)
    intensity = classify_emotion(mention, "intensity", 15)
    print(f"📈 Brand Sentiment: {sentiment}")
    print(f"💥 Impact Level: {intensity}")
    print("-" * 60)


🏢 BRAND MENTION SENTIMENT ANALYSIS:

🏢 Mention 1: Just tried the new @CoffeeShop blend and it's incredible! Perfect start to my morning.
📈 Brand Sentiment: positive
💥 Impact Level: medium
------------------------------------------------------------

🏢 Mention 2: @TechCompany's latest update broke my favorite feature. Really disappointed right now.
📈 Brand Sentiment: negative
💥 Impact Level: high
------------------------------------------------------------

🏢 Mention 3: @FitnessApp helped me reach my goals! Lost 20 pounds thanks to their program.
📈 Brand Sentiment: positive
💥 Impact Level: medium
------------------------------------------------------------

🏢 Mention 4: Had a terrible experience at @Restaurant yesterday. Food was cold and service was slow.
📈 Brand Sentiment: negative
💥 Impact Level: medium
------------------------------------------------------------

🏢 Mention 5: @BookStore has an amazing selection and the staff is always so helpful and knowledgeable.
📈 Brand Sentiment

In [28]:
complex_scenarios = [
    "I'm excited about the new job opportunity, but scared about leaving my comfort zone and starting over.",
    "Watching my daughter graduate makes me so proud, yet sad that she's growing up so fast.",
    "Finally finished the project I've been working on for months. Relief mixed with exhaustion.",
    "The vacation was amazing, but coming back to reality and work stress is hitting me hard.",
    "Happy about the promotion, but worried about the increased responsibilities and expectations."
]

print("\n🌈 COMPLEX EMOTIONAL SCENARIO ANALYSIS:")
for i, scenario in enumerate(complex_scenarios, 1):
    print(f"\n🌈 Scenario {i}: {scenario}")
    emotion = classify_emotion(scenario, "emotion", 30)
    mood = classify_emotion(scenario, "mood", 25)
    print(f"😊 Primary Emotion: {emotion}")
    print(f"🎭 Overall Mood: {mood}")
    print("-" * 60)


🌈 COMPLEX EMOTIONAL SCENARIO ANALYSIS:

🌈 Scenario 1: I'm excited about the new job opportunity, but scared about leaving my comfort zone and starting over.
😊 Primary Emotion: fear
🎭 Overall Mood: excited
------------------------------------------------------------

🌈 Scenario 2: Watching my daughter graduate makes me so proud, yet sad that she's growing up so fast.
😊 Primary Emotion: sadness
🎭 Overall Mood: sad
------------------------------------------------------------

🌈 Scenario 3: Finally finished the project I've been working on for months. Relief mixed with exhaustion.
😊 Primary Emotion: joy
🎭 Overall Mood: content
------------------------------------------------------------

🌈 Scenario 4: The vacation was amazing, but coming back to reality and work stress is hitting me hard.
😊 Primary Emotion: sadness
🎭 Overall Mood: sad
------------------------------------------------------------

🌈 Scenario 5: Happy about the promotion, but worried about the increased responsibilities and 